# 概要

20211005

本コードは画像AIのための池の画像切り出しコードである<br>

# ライブラリのインポート

## Loggerのインポート

`MyLogger`は自作のロガークラスであり、`util/MyLogger.py`で実装されている。<br>

In [4]:
%matplotlib inline

from IPython.display import display

import argparse
import os
import sys
import re
import cv2
import shutil
import time as tm
import datetime as dt
import numpy as np
import pandas as pd
from collections import deque
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from tqdm import tqdm
from pathlib import Path
from PIL import Image

from utils import MyLogger

sns.set()

# Configファイルのロード

設定情報やハイパーパラメータをjsonからロードする。<br>
引数ではないが、利便性からargsに格納する。<br>

In [5]:
config_path = "configs/check_image"
config_filename = "east_3-2_final_settling_tank_201811"

args = pd.read_json(Path(config_path, f"{config_filename}.json"), typ = 'series', encoding = "utf-8")
display(args)

seed                                                       20190517
filelist_dir      results/create_filelist/east_3-2_final_settlin...
outdir_org        E:/image_ai2021/hiroshima_seibu/east_3-2_final...
outdir_crop       E:/image_ai2021/hiroshima_seibu/east_3-2_final...
start_index                                      2018-11-1 00:00:00
end_index                                       2018-11-30 23:59:59
crop_lat_n                                                      400
crop_lat_s                                                      900
crop_lon_w                                                      400
crop_lon_e                                                      900
resize_height                                                   256
resize_width                                                    256
illum_low_lim                                                    85
illum_high_lim                                                  100
dtype: object

# アウトプットディレクトリの生成

`experiments.prepare_output_dir`はアウトプットディレクトリを生成して、モデル情報などを自動的に保存してくれる。

In [6]:
# assert isinstance(args.outdir_org, str), 'args.outdir_org must be path : type \'str\' not {}'.format(type(args.outdir_org))
# assert isinstance(args.outdir_crop, str), 'args.outdir_crop must be path : type \'str\' not {}'.format(type(args.outdir_crop))


outdir_log = Path("results", "copy_image", config_filename)
outdir_log.mkdir(parents=True, exist_ok = True)
args.to_csv(Path(outdir_log, config_filename + ".csv"))

display(f'Org files are saved in {args.outdir_org}')
display(f'Croped files are saved in {args.outdir_crop}')
display(f'Log files are saved in {outdir_log}')

'Org files are saved in E:/image_ai2021/hiroshima_seibu/east_3-2_final_settling_tank/raw'

'Croped files are saved in E:/image_ai2021/hiroshima_seibu/east_3-2_final_settling_tank/crop'

'Log files are saved in results\\copy_image\\east_3-2_final_settling_tank_201811'

# Loggerの設定

MyLoggerクラスによって汎用ロガーを作成する。<br>
標準出力にINFO情報までを出力し、`root`と`filename`に指定したファイルにDEBUG情報まで出力する。<br>
moduleNmaeは`__name__`とすることでmainを指定できる。

In [7]:
path_log = Path('logs/')
path_log.mkdir(parents = True, exist_ok = True)

from logging import ERROR, INFO
ml = MyLogger(root = path_log,
              filename = "crop_image.log",
              stream_level = INFO,
              moduleName = __name__)
logger = ml.getModuleLogger()

logger.info("start script")
logger.info(f"config file name :{config_filename}")

[2021-10-05 18:13:49,689 | __main__ | INFO] start script
[2021-10-05 18:13:49,699 | __main__ | INFO] config file name :east_3-2_final_settling_tank_201811


In [8]:
t_s = tm.time()

filelist_path = Path(args.filelist_dir, "file_list.csv")
file_list = pd.read_csv(filelist_path, index_col = "datetime", parse_dates = True)

duration = tm.time() - t_s
logger.info("get file list : elapsed time is {:.2f} [s]".format(duration))

display(file_list.shape)
display(file_list.index.dtype)
display(file_list.dtypes)
display(file_list.head())
display(file_list.tail())
display(file_list.info())

[2021-10-05 18:13:51,278 | __main__ | INFO] get file list : elapsed time is 0.74 [s]


(209569, 1)

dtype('<M8[ns]')

filename    object
dtype: object

,filename
datetime,
2018-03-02 10:40:10,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2018-08-21 09:10:46,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2018-08-21 09:10:56,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2018-08-21 09:11:06,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2018-08-21 09:11:16,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...


,filename
datetime,
2019-12-05 13:16:18,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2019-12-05 13:16:28,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2019-12-05 13:16:38,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2019-12-05 13:16:48,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2019-12-05 13:16:58,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 209569 entries, 2018-03-02 10:40:10 to 2019-12-05 13:16:58
Data columns (total 1 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   filename  209569 non-null  object
dtypes: object(1)
memory usage: 3.2+ MB


None

In [9]:
start_datetime = dt.datetime.strptime(args.start_index, '%Y-%m-%d %H:%M:%S')
file_list = file_list[file_list.index >= start_datetime]

end_datetime = dt.datetime.strptime(args.end_index, '%Y-%m-%d %H:%M:%S')
file_list = file_list[file_list.index <= end_datetime]


display(file_list.shape)    
display(file_list.head())
display(file_list.tail())

(6500, 1)

,filename
datetime,
2018-11-20 13:24:06,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2018-11-20 13:24:16,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2018-11-20 13:24:26,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2018-11-20 13:24:36,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2018-11-20 13:24:46,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...


,filename
datetime,
2018-11-30 23:54:52,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2018-11-30 23:55:02,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2018-11-30 23:55:12,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2018-11-30 23:55:22,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2018-11-30 23:55:32,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...


In [10]:
t_s = tm.time()

pick_file_list = deque(maxlen = 10**6)

for tmp_datetime, tmp_image in tqdm(file_list.iterrows()):
# for tmp_datetime, tmp_image in tqdm(file_list.sample(n = 100).iterrows()):
    outdir_org_month = Path(args.outdir_org, tmp_datetime.strftime('%Y-%m'))
    outdir_org_month.mkdir(parents=True, exist_ok=True)
    outdir_crop_month = Path(args.outdir_crop, tmp_datetime.strftime('%Y-%m'))
    outdir_crop_month.mkdir(parents=True, exist_ok=True)
    
#     image = cv2.imread(tmp_image.filename.as_posix(), cv2.IMREAD_UNCHANGED)
    
    image_pil = Image.open(tmp_image.filename)
    image_cv2 = np.asarray(image_pil)
    image = cv2.cvtColor(image_cv2, cv2.COLOR_RGB2BGR)
    
    #infanviewとはｘｙ逆
    #xが縦（上から）、yが横（ひだりから）
    image_trim = image[args.crop_lat_n:args.crop_lat_s, args.crop_lon_w:args.crop_lon_e]
    image_resize = cv2.resize(image_trim, (args.resize_height, args.resize_width))

    # hsv変換
    image_hsv = cv2.cvtColor(image_resize, cv2.COLOR_BGR2HSV)
    image_v = image_hsv[:, :, 2]
    illum = image_v.mean()

    if (illum >= args.illum_low_lim) and (illum <= args.illum_high_lim):
        pick_file_list.append([tmp_datetime, tmp_image.filename])

        shutil.copy2(tmp_image.filename,
                     Path(outdir_org_month, f"{tmp_datetime.strftime('%Y%m%d%H%M%S')}.jpg"))
        # cv2.imwrite(Path(outdir_crop_month, f"{tmp_datetime.strftime('%Y%m%d%H%M%S')}.jpg").as_posix(), image_resize)
        
        # image_pil = Image.fromarray(cv2.cvtColor(image_resize, cv2.COLOR_BGR2RGB))
        # image_pil.save(Path(outdir_crop_month, f"{tmp_datetime.strftime('%Y%m%d%H%M%S')}.jpg"), quality = 95)

        
duration = tm.time() - t_s
logger.info("crop image : elapsed time is {:.2f} [s]".format(duration))

1694it [01:07, 25.11it/s]


TypeError: Expected Ptr<cv::UMat> for argument 'src'

In [ ]:
pick_file_list_pd = pd.DataFrame(list(pick_file_list), columns = ["datetime", "filename"])
pick_file_list_pd = pick_file_list_pd.set_index("datetime")
pick_file_list_pd = pick_file_list_pd.sort_index()

display(pick_file_list_pd.head())
display(pick_file_list_pd.tail())
display(pick_file_list_pd.info())

,filename
datetime,
2019-12-02 08:43:38,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2019-12-03 06:22:18,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2019-12-03 06:51:28,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2019-12-03 06:51:38,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2019-12-03 06:51:48,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...


,filename
datetime,
2019-12-04 23:51:38,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2019-12-04 23:51:48,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2019-12-04 23:51:58,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2019-12-04 23:52:08,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2019-12-04 23:52:18,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 287 entries, 2019-12-02 08:43:38 to 2019-12-04 23:52:18
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  287 non-null    object
dtypes: object(1)
memory usage: 4.5+ KB


None

In [ ]:
pick_file_list_path = Path(outdir_log, "pick_file_list.csv")
pick_file_list_pd.to_csv(pick_file_list_path)
